In [1]:
import pickle
file = open('split/test_set.pkl', 'rb')
data = pickle.load(file)
file.close()

In [2]:
file = open('timestamp_log.pkl', 'rb')
timestamp_log = pickle.load(file)
file.close()

In [3]:
import pandas as pd
pd.DataFrame(data)

,0
0,2015-01-01 00:00:00
1,2015-01-01 00:15:00
2,2015-01-01 00:30:00
3,2015-01-01 00:45:00
4,2015-01-01 01:00:00
...,...
35035,2015-12-31 22:45:00
35036,2015-12-31 23:00:00
35037,2015-12-31 23:15:00
35038,2015-12-31 23:30:00


In [5]:
meta = pd.DataFrame.from_dict(timestamp_log, orient='index', dtype=None, columns=None)

In [6]:
#Number of expected batches for evaluation. 
len(data)/32 * 7

7665.0

In [7]:
#Number of expected datapoints:
len(data)*7

245280

In [8]:
2600/7665

0.33920417482061316

In [9]:
import numpy as np
res = np.load("epoch_results.npy")
#np.save("valid_results_conv3d_normalized_epoch4.npy",res)

In [10]:
MIN=-22
res2 = (res*1300)+MIN
x = pd.DataFrame(res2)

In [11]:
colonnes = {0:"out_ghi0",1:"out_ghi1",2:"out_ghi3",3:"out_ghi6",
           4:"tar_ghi0",5:"tar_ghi1",6:"tar_ghi3",7:"tar_ghi6"}

In [12]:
données = x[x[0]!=MIN].rename(columns=colonnes)
données

,out_ghi0,out_ghi1,out_ghi3,out_ghi6,tar_ghi0,tar_ghi1,tar_ghi3,tar_ghi6
0,169.342182,367.085603,716.707566,890.917101,154.661523,368.355405,748.471728,979.039726
1,129.436921,253.159954,373.824486,204.420926,22.553421,46.462256,83.654499,86.858769
2,25.414893,167.809993,379.782581,344.055512,3.834056,30.061155,61.377796,58.846346
3,13.384711,155.924041,421.791443,687.988064,6.105697,45.496306,258.943725,546.915471
4,301.549423,142.945769,9.446236,-2.211470,296.535669,130.832638,-3.120753,-3.353912
...,...,...,...,...,...,...,...,...
108648,55.932464,11.304323,7.724032,5.945107,25.251412,-0.909070,-1.695151,-2.594479
108649,114.988594,42.454562,3.342545,5.489712,23.299534,16.511266,-2.454584,-2.794331
108650,167.949661,71.468076,3.925027,10.994601,249.590695,57.074332,-2.667758,-2.727713
108651,155.595985,127.077801,39.007886,3.749918,338.058054,178.870137,69.072043,-3.214017


In [13]:
def rmse(données):
    rmse_t0 = np.sqrt(((données["out_ghi0"] - données["tar_ghi0"])**2).sum()/len(données))
    rmse_t1 = np.sqrt(((données["out_ghi1"] - données["tar_ghi1"])**2).sum()/len(données))
    rmse_t3 = np.sqrt(((données["out_ghi3"] - données["tar_ghi3"])**2).sum()/len(données))
    rmse_t6 = np.sqrt(((données["out_ghi6"] - données["tar_ghi6"])**2).sum()/len(données))
    print(rmse_t0, rmse_t1, rmse_t3, rmse_t6)
    print("mean=", np.mean([rmse_t0, rmse_t1, rmse_t3, rmse_t6]))

In [14]:
rmse(données)

94.196803589831 106.8453808211193 123.95675151067603 115.53846004030994
mean= 110.13434899048407


,0,1,2,3,4,5,6,7,out_ghi0,out_ghi1,out_ghi3,out_ghi6,tar_ghi0,tar_ghi1,tar_ghi3,tar_ghi6


In [36]:
données.head(1)

,out_ghi0,out_ghi1,out_ghi3,out_ghi6,tar_ghi0,tar_ghi1,tar_ghi3,tar_ghi6
0,169.342182,367.085603,716.707566,890.917101,154.661523,368.355405,748.471728,979.039726


In [33]:
meta["cle"] = (meta[4].round(4).astype(str) + " - " + meta[5].round(4).astype(str) + " - " + meta[6].round(4).astype(str)+ " - " + meta[7].round(4).astype(str))

In [37]:
données["cle"] = (données["tar_ghi0"].round(4).astype(str) + " - " + données["tar_ghi1"].round(4).astype(str) + " - " + données["tar_ghi3"].round(4).astype(str)+ " - " + données["tar_ghi6"].round(4).astype(str))

In [49]:
merged = pd.DataFrame.merge(meta.reset_index(), données, left_index=True, right_index=True)
merged[["cle_x", "cle_y"]]

,cle_x,cle_y
0,154.3733 - 368.2267 - 748.6267 - 979.3667,154.6615 - 368.3554 - 748.4717 - 979.0397
1,22.1667 - 46.0933 - 83.3133 - 86.52,22.5534 - 46.4623 - 83.6545 - 86.8588
2,3.4333 - 29.68 - 61.02 - 58.4867,3.8341 - 30.0612 - 61.3778 - 58.8463
3,5.7067 - 45.1267 - 258.7333 - 546.92,6.1057 - 45.4963 - 258.9437 - 546.9155
4,296.3533 - 130.5267 - -3.5267 - -3.76,296.5357 - 130.8326 - -3.1208 - -3.3539
...,...,...
108648,24.8667 - -1.3133 - -2.1 - -3.0,25.2514 - -0.9091 - -1.6952 - -2.5945
108649,22.9133 - 16.12 - -2.86 - -3.2,23.2995 - 16.5113 - -2.4546 - -2.7943
108650,249.3733 - 56.7133 - -3.0733 - -3.1333,249.5907 - 57.0743 - -2.6678 - -2.7277
108651,337.9067 - 178.6 - 68.72 - -3.62,338.0581 - 178.8701 - 69.072 - -3.214
